In [1]:
ds_path='../../../data/dataset/lc-quad-wikidata-2021-10-03'

In [2]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
from tqdm import tqdm

In [3]:
raw_datasets = load_dataset("lc_quad")

Using custom data configuration default
Reusing dataset lc_quad (/home/shyaz/.cache/huggingface/datasets/lc_quad/default/2.0.0/139ee1f12aca006669dcc1f282ec02e126c69e7595453db443ab022643d54086)


In [4]:
raw_datasets["train"][0]

{'NNQT_question': 'What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}',
 'paraphrased_question': "What is Delta Air Line's periodical literature mouthpiece?",
 'question': 'What periodical literature does Delta Air Lines use as a moutpiece?',
 'sparql_dbpedia18': 'select distinct ?obj where { ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#subject> <http://wikidata.dbpedia.org/resource/Q188920> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate> <http://www.wikidata.org/entity/P2813> . ?statement <http://www.w3.org/1999/02/22-rdf-syntax-ns#object> ?obj . ?obj <http://www.wikidata.org/entity/P31> <http://wikidata.dbpedia.org/resource/Q1002697> } ',
 'sparql_wikidata': ' select distinct ?obj where { wd:Q188920 wdt:P2813 ?obj . ?obj wdt:P31 wd:Q1002697 } ',
 'subgraph': 'simple question right',
 'template': ' <S P ?O ; ?O instanceOf Type>',
 'template_index': 65,
 'uid': 19719}

In [5]:
rep_dict = {"ASK": "ask", "WHERE": "where", "SELECT": "select", "{": "[","}": "]", }
rep_dict2 = {"{": "","}": "" }
def replace_all(text, dict):
    for i, j in dict.items(): text = text.replace(i, j)
    return text

## Goal: Trim + Replace LowerCase + Remove weirdly long Question + replaceQ/P

In [7]:
import pandas as pd
import re
df_q = pd.read_csv('../../../data/kdwd/q_19174151.csv').set_index('item_id')
df_p = pd.read_csv('../../../data/kdwd/p_12267.csv').set_index('property_id')

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

def encode_props(qry):
    # Q
    for m in re.finditer(":Q\d+", qry):
        x = m.group(0)[1:]
        newstring = df_q.loc[int(x[1:])]['en_label'].iloc[0].replace(" ", "_").replace("-", "_")
        qry = qry.replace(x, newstring)
    # P
    for m in re.finditer(":P\d+", qry):
        x = m.group(0)[1:]
        newstring = df_p.loc[int(x[1:])]['en_label'].iloc[0].replace(" ", "_").replace("-", "_")
        qry = qry.replace(x, newstring)
    return qry
# Test
encode_props('SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s ps:P1082 ?obj . ?s pq:P585 ?x filter(contains(YEAR(?x),\'2009\')) }')
# "select ?obj where [ wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) ]"

"SELECT ?obj WHERE { wd:somalia p:population ?s . ?s ps:population ?obj . ?s pq:point_in_time ?x filter(contains(YEAR(?x),'2009')) }"

In [8]:
encode_props('select ?obj where { wd:Q124057 p:P1411 ?s . ?s ps:P1411 ?obj . ?s pq:P1686 wd:Q3915489 }')

'select ?obj where { wd:dolores_del_río p:nominated_for ?s . ?s ps:nominated_for ?obj . ?s pq:for_work wd:otra }'

In [9]:
def prepare(ds):
    col = 'translation'
    df = pd.DataFrame(columns=[col])
    for d in tqdm(ds):
        try:
            qry = replace_all(d['sparql_wikidata'], rep_dict).strip()
            qry = encode_props(qry)
            if len(d['question'])<250: 
                df = df.append({col: {'en':d['question'], 'sparql': qry}}, ignore_index=True)
            if d['paraphrased_question']!=[] and len(d['paraphrased_question'])<250:
                df = df.append({col: {'en':d['paraphrased_question'], 'sparql': qry}}, ignore_index=True)
        except: pass
    return df

In [10]:
df_test = prepare(raw_datasets["test"])

100%|██████████| 4781/4781 [02:29<00:00, 32.07it/s]


In [11]:
df_train = prepare(raw_datasets["train"])

100%|██████████| 19293/19293 [08:36<00:00, 37.36it/s]


In [12]:
df_all = pd.concat([df_train, df_test])
# shuffling all
df_all  = df_all.sample(frac = 1)
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_all, test_size=0.2)

In [13]:
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [17]:
pd.options.display.max_colwidth = 100
df_test.head()

,translation
480,"{'en': 'When Alec Guinness was nominated as Academy Award for Best Actor?', 'sparql': 'select ?v..."
11751,"{'en': 'Let me know financial marker whose title has the word soil in it.', 'sparql': 'select DI..."
10,{'en': 'Tell me the historical period which is part of the time period of Muromachi period and w...
1071,"{'en': 'Warren G. Harding, whose electoral district is Ohio, holds what position?', 'sparql': 's..."
13148,"{'en': 'Where did Lyudmila Pavlinchenko begin school at in 1937?', 'sparql': 'select ?obj where ..."


In [20]:
df_test.iloc[0].translation    

{'en': 'When Alec Guinness was nominated as Academy Award for Best Actor?',
 'sparql': 'select ?value where [ wd:alec_guinness p:nominated_for ?s . ?s ps:nominated_for wd:best_actor . ?s pq:point_in_time ?value]'}

In [14]:
ds_train

Dataset({
    features: ['translation', '__index_level_0__'],
    num_rows: 15123
})

In [15]:
mother_ds = DatasetDict({'train': ds_train, 'test':ds_test})

In [16]:
mother_ds.save_to_disk(ds_path)